In [ ]:
#%%capture #Там при установке просит клавишу нажать
!!add-apt-repository ppa:libreoffice/ppa && sudo apt-get update
!!apt-get install unoconv

In [2]:
#!!libreoffice --convert-to pdf --outdir \. input.doc 

In [6]:
import sys
import subprocess
import re

In [7]:
def convert_to_pdf(in_file, path=''):
    args = ['libreoffice', '--headless', '--convert-to', 'pdf', '--outdir', path, in_file]
    process = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=None)
    #print(process.stdout.decode())
    filename = re.search('-> (.pdf) using filter', process.stdout.decode())
    return filename is None

In [8]:
convert_to_pdf('input.rtf', 'out')

True

In [9]:
%%capture
!!pip install --upgrade --force-reinstall fitz
!!pip install --upgrade --force-reinstall pymupdf
!!pip install webcolors

In [10]:
import os
import fitz
from webcolors import name_to_rgb

In [11]:
def anonymize_pdf(in_file, out_file, text, color='green', filled=True):
    pix = fitz.Pixmap(fitz.csRGB, (0, 0, 300, 300), 0)
    pix.set_rect(pix.irect, name_to_rgb(color))
    doc = fitz.open(in_file)
    for page in doc:
        text_instances = page.search_for(text)
        for inst in text_instances:
            highlight = page.add_highlight_annot(inst)
            page.insert_image(highlight.rect, pixmap=pix, keep_proportion=False, overlay=filled) 
    doc.save(out_file, garbage=4, deflate=True, clean=True)
    return os.path.exists(out_file)

In [13]:
anonymize_pdf(in_file='out/input.pdf', out_file='output.pdf', text='Вопрос', filled=True)

True